In [1]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

In [2]:
def preprocess_images(folder, label, image_size=(224, 224)):
    data = []
    labels = []

    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, image_size)
        img = img / 255.0  # Normalize pixel values to the range [0, 1]

        data.append(img)
        labels.append(label)

    return np.array(data), np.array(labels)

In [3]:
dataset_pos = 'datasets/brain_mri_scan_images/positive'
dataset_neg = 'datasets/brain_mri_scan_images/negative'

positive_data, positive_labels = preprocess_images(dataset_pos, 1)
negative_data, negative_labels = preprocess_images(dataset_neg, 0)

x = np.concatenate((positive_data, negative_data))
y = np.concatenate((positive_labels, negative_labels))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# print(x_train, x_test)
# print(y_train, y_test)

print(f'Train: X={x_train.shape}, y={y_train.shape}')
print(f'Test: X={x_test.shape}, y={y_test.shape}')

input_shape = x_train.shape[1:4]
print(input_shape)


Train: X=(172, 224, 224, 1), y=(172,)
Test: X=(44, 224, 224, 1), y=(44,)
(224, 224, 1)


In [4]:
from tensorflow import keras
from keras import layers

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(1, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 186624)            0         
                                                                 
 dropout (Dropout)           (None, 186624)            0